# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [10]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [11]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head()

,split,metric,value,experiment,dataset
0,train,eer,0.059028,lj_finetune_facebook_wav2vec2-base-960h,rav
1,val,eer,0.312500,lj_finetune_facebook_wav2vec2-base-960h,rav
2,test,eer,0.345833,lj_finetune_facebook_wav2vec2-base-960h,rav
3,train,beer_Happiness,0.215278,lj_finetune_facebook_wav2vec2-base-960h,rav
4,train,beer_Anger,0.000000,lj_finetune_facebook_wav2vec2-base-960h,rav


In [12]:
from IPython.display import Markdown, display_html

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = global_only_metrics.query("split == @split").pivot(
        index="experiment", columns=["dataset", "metric"], values="value"
    )

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = per_emo_metrics.query("split == @split").pivot(
        index="experiment", columns=["dataset", "emotion"], values="value"
    )

    # print
    display(Markdown(f"# {split}"))
    display(global_pivot_metrics)
    display(per_emo_pivot_metrics)
    # display_html(
    #     global_pivot_metrics.style.set_table_attributes(
    #         "style='display:inline'"
    #     )._repr_html_()
    #     + per_emo_pivot_metrics.style.set_table_attributes(
    #         "style='display:inline'"
    #     )._repr_html_(),
    #     raw=True,
    # )

# train

dataset                                             rav                 iem  \
metric                                              eer      beer       eer   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.500000  0.500000  0.454254   
lj_featureextract_facebook_wav2vec2-large      0.537037  0.537037  0.471910   
lj_finetune_facebook_wav2vec2-base-960h        0.059028  0.059028  0.364901   
lj_finetune_facebook_wav2vec2-large            0.008102  0.008102  0.291600   

dataset                                                  
metric                                             beer  
experiment                                               
lj_featureextract_facebook_wav2vec2-base-960h  0.494325  
lj_featureextract_facebook_wav2vec2-large      0.515743  
lj_finetune_facebook_wav2vec2-base-960h        0.399947  
lj_finetune_facebook_wav2vec2-large            0.297666

dataset                                             rav                      \
emotion                                       Happiness     Anger   Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.798611  0.444444  0.409722   
lj_featureextract_facebook_wav2vec2-large      0.833333  0.444444  0.354167   
lj_finetune_facebook_wav2vec2-base-960h        0.215278  0.000000  0.062500   
lj_finetune_facebook_wav2vec2-large            0.006944  0.000000  0.020833   

dataset                                                                      \
emotion                                            Fear   Disgust  Surprise   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.402778  0.569444  0.375000   
lj_featureextract_facebook_wav2vec2-large      0.395833  0.743056  0.451389   
lj_finetune_facebook_wav2vec2-base-960h        0.006944  0.041667  0.027778   
lj_finetune_facebook_wav2vec2-large            0.006944  0.006944  0.006944   

dataset                                             iem                      \
emotion                                       Happiness     Anger   Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.662338  0.176183  0.696970   
lj_featureextract_facebook_wav2vec2-large      0.675966  0.186992  0.750769   
lj_finetune_facebook_wav2vec2-base-960h        0.350840  0.105351  0.590909   
lj_finetune_facebook_wav2vec2-large            0.638655  0.063545  0.178788   

dataset                                                  
emotion                                         Neutral  
experiment                                               
lj_featureextract_facebook_wav2vec2-base-960h  0.441810  
lj_featureextract_facebook_wav2vec2-large      0.449244  
lj_finetune_facebook_wav2vec2-base-960h        0.552688  
lj_finetune_facebook_wav2vec2-large            0.309677

# val

dataset                                             rav                 iem  \
metric                                              eer      beer       eer   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.541667  0.541667  0.509615   
lj_featureextract_facebook_wav2vec2-large      0.645833  0.645833  0.528846   
lj_finetune_facebook_wav2vec2-base-960h        0.312500  0.312500  0.442308   
lj_finetune_facebook_wav2vec2-large            0.166667  0.166667  0.394231   

dataset                                                  
metric                                             beer  
experiment                                               
lj_featureextract_facebook_wav2vec2-base-960h  0.532035  
lj_featureextract_facebook_wav2vec2-large      0.540207  
lj_finetune_facebook_wav2vec2-base-960h        0.493338  
lj_finetune_facebook_wav2vec2-large            0.410118

dataset                                             rav                        \
emotion                                       Happiness  Anger Sadness   Fear   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h     0.875  0.250   0.125  0.375   
lj_featureextract_facebook_wav2vec2-large         1.000  0.375   0.125  0.625   
lj_finetune_facebook_wav2vec2-base-960h           0.750  0.125   0.125  0.500   
lj_finetune_facebook_wav2vec2-large               0.000  0.125   0.000  0.500   

dataset                                                              iem  \
emotion                                       Disgust Surprise Happiness   
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h   0.750    0.875  0.723077   
lj_featureextract_facebook_wav2vec2-large       0.875    0.875  0.754098   
lj_finetune_facebook_wav2vec2-base-960h         0.125    0.250  0.509804   
lj_finetune_facebook_wav2vec2-large             0.125    0.250  0.745098   

dataset                                                                      
emotion                                           Anger   Sadness   Neutral  
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h  0.193548  0.764706  0.446809  
lj_featureextract_facebook_wav2vec2-large      0.283333  0.769231  0.354167  
lj_finetune_facebook_wav2vec2-base-960h        0.207792  0.647059  0.608696  
lj_finetune_facebook_wav2vec2-large            0.181818  0.235294  0.478261

# test

dataset                                             rav                 iem  \
metric                                              eer      beer       eer   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h  0.525000  0.525000  0.525490   
lj_featureextract_facebook_wav2vec2-large      0.541667  0.541667  0.523529   
lj_finetune_facebook_wav2vec2-base-960h        0.345833  0.345833  0.390196   
lj_finetune_facebook_wav2vec2-large            0.275000  0.275000  0.352941   

dataset                                                  
metric                                             beer  
experiment                                               
lj_featureextract_facebook_wav2vec2-base-960h  0.520835  
lj_featureextract_facebook_wav2vec2-large      0.519301  
lj_finetune_facebook_wav2vec2-base-960h        0.415085  
lj_finetune_facebook_wav2vec2-large            0.345363

dataset                                             rav                        \
emotion                                       Happiness  Anger Sadness   Fear   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h     0.750  0.325   0.425  0.625   
lj_featureextract_facebook_wav2vec2-large         0.875  0.300   0.225  0.550   
lj_finetune_facebook_wav2vec2-base-960h           0.675  0.150   0.300  0.375   
lj_finetune_facebook_wav2vec2-large               0.250  0.225   0.100  0.450   

dataset                                                              iem  \
emotion                                       Disgust Surprise Happiness   
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h   0.500    0.525  0.728395   
lj_featureextract_facebook_wav2vec2-large       0.750    0.550  0.728395   
lj_finetune_facebook_wav2vec2-base-960h         0.350    0.225  0.370370   
lj_finetune_facebook_wav2vec2-large             0.475    0.150  0.574074   

dataset                                                                      
emotion                                           Anger   Sadness   Neutral  
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h  0.187050  0.714286  0.453608  
lj_featureextract_facebook_wav2vec2-large      0.158273  0.767857  0.422680  
lj_finetune_facebook_wav2vec2-base-960h        0.100719  0.642857  0.546392  
lj_finetune_facebook_wav2vec2-large            0.107914  0.276786  0.422680